In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns=None
pd.options.mode.chained_assignment = None

In [2]:
triggers_actions = pd.read_csv(r'/home/kusapochka/triggers/triggers_actions.csv')

In [3]:
triggers_actions['date'] = pd.to_datetime(triggers_actions['date'], errors='coerce')
triggers_actions['action_date'] = pd.to_datetime(triggers_actions['action_date'], errors='coerce')

In [ ]:
triggers_actions.info()

In [ ]:
triggers_actions.head(10)

In [6]:
# подготовим признаки, чтобы далее оценить какие типы взаимодействий или комбинации тригеров приводят к более успешному результату 

In [4]:
# сортируем по пользователю и дате
triggers_actions = triggers_actions.sort_values(by=['guid', 'date'])

### признак - время с поледнего взаимодействия 

In [5]:
# добавим в колонку 'last_action_date' смещённое время взаимодействия, чтобы дальше отталкиваться от этого времени, 
# как последнего, когда было произведено взаимодействие
triggers_actions['last_action_date'] = triggers_actions.groupby('guid')['action_date'].shift()

In [6]:
# теперь по каждому пользователю заполним пропуски этой самой датой, ведь пропуск обозначает, что взаимодействия не было, 
# тогда эта дата(до новой) будет для этих тригеров последней, когда было взаимодействие
# важный момент с самой строкой, когды было взаимодействие нам тоже нужно заполнить её датой прошлого взаимодействия,
# чтобы посмотреть сколько времени было до последнего взаимодействия, для этого и делался shift  
triggers_actions['last_action_date'] = triggers_actions.groupby('guid')['last_action_date'].ffill()

In [ ]:
# теперь собственно посчитаем время между текущим моментов и датой последнего действия 
# пропуски обозначают, что в нашем временном отрезке взаимодействия не было, тогда будем считать для 
# каждого момента - время от первого тригера 
def time_since_last_action(group):
    # вычисляем минимальную дату для группы
    date_min = group['date'].min()

    # если есть значение в 'last_action_date', считаем разницу между текущей и последней датой
    # если нет, считаем разницу между текущей датой и минимальной датой группы
    group['time_since_last_action'] = group.apply(
        lambda row: (row['date'] - row['last_action_date']).total_seconds() / (3600 * 24)
        if pd.notna(row['last_action_date'])
        else (row['date'] - date_min).total_seconds() / (3600 * 24),
        axis=1
    )
    return group

# применяем функцию к сгруппированным данным по 'guid'
triggers_actions = triggers_actions.groupby('guid', group_keys=False).apply(time_since_last_action)

In [ ]:
triggers_actions.to_csv(r'C:\Users\User\triggers\time_since_last_action.csv',index = False)

In [ ]:
# отрицательное время появилось скорее всего из-за неудаленных дубликатов, все эти строки с взаимодействием 
# проверим, если это так, то просто удалим строки с отриц значениями

### признак - тип последнего удачного взаимодействия

In [2]:
triggers_actions = pd.read_csv(r'C:\Users\User\triggers\time_since_last_action.csv')

In [3]:
# создаем колонку для хранения типа последнего успешного взаимодействия, по умолчанию None
triggers_actions['last_successful_type'] = None

In [ ]:

# находим тип успешного действия и сдвигаем его на одну строку вниз
triggers_actions['last_successful_type'] = triggers_actions.groupby('guid').apply(
    lambda group: group['type'].where(group['result'] == 1).shift(1)
).reset_index(level=0, drop=True)

In [ ]:
triggers_actions[triggers_actions['result'] == 1].head(5)

In [13]:
one= triggers_actions[triggers_actions['guid'] == '018783be-25da-710a-36a7-2d50f5a61395'] 

In [ ]:
one

In [ ]:
triggers_actions['last_successful_type'].value_counts()

In [ ]:
triggers_actions['last_successful_type']

In [12]:
# распространяем последний успешный тип на все последующие строки внутри guid
triggers_actions['last_successful_type'] = triggers_actions.groupby('guid')['last_successful_type'].ffill()

In [15]:
triggers_actions.to_csv(r'C:\Users\User\triggers\time_since_last_action_type.csv',index = False)

In [ ]:
trigers_actions_save = triggers_actions.drop(columns = 'triggers_before_next_action')

In [ ]:
triggers_actions.head(3)

### добавляем признак - количество тригеров с момента последнего взаимодействия или за последние 14 дней

In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns=None
pd.options.mode.chained_assignment = None

In [2]:
triggers_actions_check = pd.read_csv(r'/home/kusapochka/triggers/time_since_last_action_type.csv')

In [3]:
# создаем маску для строк с действием
action_mask = triggers_actions_check['action_date'].notna()

# создаем сдвинутую маску действий для каждой группы
triggers_actions_check['shifted_action_mask'] = triggers_actions_check.groupby('guid')['action_date'].transform(lambda x: x.notna().shift(1, fill_value=False))

# обнуляем счётчик на строках, следующих за действием
triggers_actions_check.loc[triggers_actions_check['shifted_action_mask'], 'trigger_counter'] = 0


In [ ]:
triggers_actions_check.head(5)

In [4]:
# Создаем маску для нулей
zero_mask = triggers_actions_check['trigger_counter'] == 0

# Заполняем NaN для удобства подсчета
triggers_actions_check['temp_counter'] = triggers_actions_check['trigger_counter'].fillna(-1)

In [5]:

# Считаем кумулятивную сумму количества строк между нулями
triggers_actions_check['cumsum'] = triggers_actions_check.groupby('guid')['temp_counter'].transform(lambda x: (x == 0).cumsum())

In [6]:

# Теперь считаем количество строк после последнего нуля в каждой группе
triggers_actions_check['count_from_zero'] = triggers_actions_check.groupby(['guid', 'cumsum']).cumcount()

In [7]:

# Восстанавливаем нули на строках, где был 0
triggers_actions_check['count_from_zero'] = triggers_actions_check['count_from_zero'].where(~zero_mask, 0)

In [8]:

# Добавляем 1, чтобы счет начинался с 1 для строк после нуля
triggers_actions_check['count_from_zero'] = triggers_actions_check['count_from_zero'] + 1



In [10]:
triggers_actions_check.drop(columns = ['last_action_date','shifted_action_mask','trigger_counter','temp_counter','cumsum'],inplace = True)
triggers_actions_check = triggers_actions_check.rename(columns = {'count_from_zero':'triggers_since_last_action'})
triggers_actions_check.head(3)

,guid,date,trigger,type,action_date,result,time_since_last_action,last_successful_type,triggers_since_last_action
0,0187808e-d664-cc1d-403d-b93ac1365ee8,2024-05-20 15:30:28,59,1,NaN,NaN,0.00000,NaN,1
1,0187808e-d664-cc1d-403d-b93ac1365ee8,2024-05-23 07:04:59,59,1,NaN,NaN,2.64897,NaN,2
2,0187808e-e508-2486-7385-30069e7c0a7f,2024-05-21 18:45:45,12,1,NaN,NaN,0.00000,NaN,1


In [12]:
triggers_actions_check.to_csv(r'/home/kusapochka/triggers/triggers_features.csv',index = False)

In [9]:
# Вывод результата
triggers_actions_check[['guid', 'trigger_counter', 'last_action_date','count_from_zero']].head(500)


,guid,trigger_counter,last_action_date,count_from_zero
0,0187808e-d664-cc1d-403d-b93ac1365ee8,NaN,NaN,1
1,0187808e-d664-cc1d-403d-b93ac1365ee8,NaN,NaN,2
2,0187808e-e508-2486-7385-30069e7c0a7f,NaN,NaN,1
3,0187808e-e508-2486-7385-30069e7c0a7f,NaN,NaN,2
4,0187808e-e508-2486-7385-30069e7c0a7f,NaN,NaN,3
5,0187808e-e508-2486-7385-30069e7c0a7f,NaN,NaN,4
6,0187808e-e508-2486-7385-30069e7c0a7f,NaN,NaN,5
7,0187808e-e508-2486-7385-30069e7c0a7f,NaN,NaN,6
8,0187808e-e508-2486-7385-30069e7c0a7f,NaN,NaN,7
9,0187808e-e508-2486-7385-30069e7c0a7f,NaN,NaN,8
